In [148]:
import sys  
sys.path.insert(0, './../')
import pandas as pd
import numpy as np
from collections import Counter
import itertools
import evaluation.evaluate as evaluate

# Load Predictions and decide for label by majority vote

In [151]:
predictionFiles = ['../submissions/submission01/LP1d_PredictionChallenge.csv', '../submissions/submission03/2d_PredictionChallenge_mel.csv', '../predictions/jannes/secondPredictionChallengeWithSecondLikely.csv']
mainPrediction = 2
#predictionFiles = ['../predictions/jannes/LP1d_Prediction_mel_pp_test.csv', "./../data/test.csv"]
#mainPrediciton = 0
predictions = [pd.read_csv(predFile) for predFile in predictionFiles]

In [53]:
predictions[0].head()

,filename,onset,offset,event_label
0,10001_mix.wav,2.880750,3.851437,Cheering
1,10001_mix.wav,5.479687,6.450375,Fireworks
2,10002_mix.wav,0.438375,3.475687,Fireworks
3,10002_mix.wav,5.135250,6.074625,Bark
4,10002_mix.wav,7.608937,8.798813,Fireworks


In [90]:
def findOverlap(otherRows, onset, offset):
    overlappingSequences = []
    for index, otherRow in otherRows.iterrows():
        if otherRow.onset<=offset and otherRow.offset>=onset and offset-onset>0:
            overlapOnset = otherRow.onset if otherRow.onset>onset else onset
            overlapOffset = otherRow.offset if otherRow.offset<offset else offset
            overlap = (overlapOffset-overlapOnset)/(offset-onset)
            overlappingSequences.append([overlapOnset, overlapOffset, overlap, otherRow.event_label])
    return overlappingSequences

In [152]:
newPrediction = []
secondChoice = []
for filename in predictions[mainPrediction].filename.unique():
    fileRows = [df.loc[df['filename'] == filename] for df in predictions]
    for index, sequence in fileRows[mainPrediction].iterrows():
        overlapping = [findOverlap(other, sequence.onset, sequence.offset) for other in fileRows]
        overlapping = list(itertools.chain(*overlapping))
        labels = [overlappingSequence[-1] for overlappingSequence in overlapping]
        occurenceCounter = Counter(labels).most_common(5)
        sumOfLabels = sum([occ[1] for occ in occurenceCounter])
        if len(occurenceCounter)>0:
            if occurenceCounter[0][1]>1:
                newPrediction.append([sequence.filename, sequence.onset, sequence.offset, occurenceCounter[0][0]])
            else:
                newPrediction.append([sequence.filename, sequence.onset, sequence.offset, sequence.event_label])
        if len(occurenceCounter)>1:
            occurenceCounter = [(occ[0], occ[1]/sumOfLabels) for occ in occurenceCounter]
            secondChoice.append([sequence.filename, sequence.onset, sequence.offset, occurenceCounter[1:]])

In [153]:
majorityVoteDf = pd.DataFrame(data=connectedPrediction, columns=['filename', 'onset', 'offset', 'event_label'])

# Connect short neighboring sequences with same label

In [154]:
connectedPrediction = []
lastSequence = [""]
maxSequenceLength = 2.2
for sequence in newPrediction:
    if lastSequence[0]==sequence[0] and lastSequence[3]==sequence[3] and sequence[2]-lastSequence[1]<maxSequenceLength:
        lastSequence = [sequence[0], lastSequence[1], sequence[2], sequence[3]]
        continue
    if len(lastSequence)==4:
        connectedPrediction.append(lastSequence) 
    lastSequence = sequence

In [155]:
connectedDf = pd.DataFrame(data=connectedPrediction, columns=['filename', 'onset', 'offset', 'event_label'])

In [163]:
connectedDf.loc[connectedDf['filename'] == "11202_mix.wav"]

,filename,onset,offset,event_label
3918,11202_mix.wav,2.506266,4.511278,Shatter
3919,11202_mix.wav,5.388471,5.789474,Bark
3920,11202_mix.wav,6.491228,7.593985,Bark


# Add multiple labels for each event

In [ ]:
multipleEvents = connectedPrediction.copy()
secondChoiceDf = pd.DataFrame(data=secondChoice, columns=['filename', 'onset', 'offset', 'event_label'])
for filename in predictions[mainPrediction].filename.unique():
    predictedLabels = connectedDf.loc[connectedDf['filename'] == filename].event_label.values
    secondChoiceRows = secondChoiceDf.loc[secondChoiceDf['filename'] == filename]
    for index, row in secondChoiceRows.iterrows():
        for label in row.event_label:
            if label[0] not in predictedLabels:
                multipleEvents.append([row.filename, row.onset, row.offset, label[0]])
multipleEventsDf = pd.DataFrame(data=multipleEvents, columns=['filename', 'onset', 'offset', 'event_label'])

In [161]:
multipleEventsDf.values[-2]

array(['12999_mix.wav', 4.5363408521303255, 5.889724310776942, 'Meow'],
      dtype=object)

In [159]:
multipleEventsDf.to_csv("../predictions/jannes/majorityVoteChallengeMultiple.csv", index=False)

# Test-Daten

In [149]:
multipleEventsDf.to_csv("../predictions/jannes/majorityVoteTestMultiple.csv", index=False)
print("Score was: ")
print(evaluate.evaluate(predictionFiles[0], "./../data/test.csv"))
print("Score now is: ")
print(evaluate.evaluate("../predictions/jannes/majorityVoteTestMultiple.csv", "./../data/test.csv"))

Score was: 
(0.831134109757114, 'Valid file')
Score now is: 
(0.9709034523809524, 'Valid file')
